In [3]:
import pandas as pd

df = pd.read_parquet(
    "hf://datasets/dvilasuero/imgsys-augmented-v5/data/train-00000-of-00001.parquet"
)

/Users/ben/code/data-is-better-together/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df

,prompt,quality_generation,style_generation,complexity_generation,image_original_schnell,grouped_model_name,image_quality_schnell,image_style_schnell,image_complexity_schnell,image_original_dev,image_quality_dev,image_style_dev,image_complexity_dev,distilabel_metadata
0,intelligence,"intelligent robot with intricate details, high...","intelligence, cyberpunk cityscape, neon lights...","intelligence manifesting as a luminous, multi-...",{'path': 'artifacts/image_generation_0/images/...,"[black-forest-labs/FLUX.1-schnell, black-fores...",{'path': 'artifacts/image_generation_1/images/...,{'path': 'artifacts/image_generation_2/images/...,{'path': 'artifacts/image_generation_3/images/...,{'path': 'artifacts/image_generation_4/images/...,{'path': 'artifacts/image_generation_5/images/...,{'path': 'artifacts/image_generation_6/images/...,{'path': 'artifacts/image_generation_7/images/...,{'raw_input_image_generation_0': {'prompt': 'i...
1,"A raccoon wearing formal clothes, wearing a to...","A raccoon in formal attire, donning a tophat a...","A raccoon in formal attire, donning a tophat a...","A raccoon in formal attire, donning a top hat ...",{'path': 'artifacts/image_generation_0/images/...,"[black-forest-labs/FLUX.1-schnell, black-fores...",{'path': 'artifacts/image_generation_1/images/...,{'path': 'artifacts/image_generation_2/images/...,{'path': 'artifacts/image_generation_3/images/...,{'path': 'artifacts/image_generation_4/images/...,{'path': 'artifacts/image_generation_5/images/...,{'path': 'artifacts/image_generation_6/images/...,{'path': 'artifacts/image_generation_7/images/...,{'raw_input_image_generation_0': {'prompt': 'A...
2,a wood cabin,"a cozy wood cabin with a stone chimney, surrou...","a wood cabin, impressionist painting, soft nat...","a quaint wood cabin nestled in a dense, ancien...",{'path': 'artifacts/image_generation_0/images/...,"[black-forest-labs/FLUX.1-schnell, black-fores...",{'path': 'artifacts/image_generation_1/images/...,{'path': 'artifacts/image_generation_2/images/...,{'path': 'artifacts/image_generation_3/images/...,{'path': 'artifacts/image_generation_4/images/...,{'path': 'artifacts/image_generation_5/images/...,{'path': 'artifacts/image_generation_6/images/...,{'path': 'artifacts/image_generation_7/images/...,{'raw_input_image_generation_0': {'prompt': 'a...
3,A tornado made of sharks crashing into a skysc...,"A tornado of sharks, each with fierce, detaile...",A tornado of sharks colliding with a skyscrape...,"A tornado composed of sharks, each with fracta...",{'path': 'artifacts/image_generation_0/images/...,"[black-forest-labs/FLUX.1-schnell, black-fores...",{'path': 'artifacts/image_generation_1/images/...,{'path': 'artifacts/image_generation_2/images/...,{'path': 'artifacts/image_generation_3/images/...,{'path': 'artifacts/image_generation_4/images/...,{'path': 'artifacts/image_generation_5/images/...,{'path': 'artifacts/image_generation_6/images/...,{'path': 'artifacts/image_generation_7/images/...,{'raw_input_image_generation_0': {'prompt': 'A...
4,a beach with apartment buildings next to it,a serene beach with apartment buildings in the...,a beach with apartment buildings in the style ...,"a serene beach with modern, eco-friendly apart...",{'path': 'artifacts/image_generation_0/images/...,"[black-forest-labs/FLUX.1-schnell, black-fores...",{'path': 'artifacts/image_generation_1/images/...,{'path': 'artifacts/image_generation_2/images/...,{'path': 'artifacts/image_generation_3/images/...,{'path': 'artifacts/image_generation_4/images/...,{'path': 'artifacts/image_generation_5/images/...,{'path': 'artifacts/image_generation_6/images/...,{'path': 'artifacts/image_generation_7/images/...,{'raw_input_image_generation_0': {'prompt': 'a...
